In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly as plt
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2022/data.csv


# Exploratory analysis of TPS June 22
First draft as of 1 June - work to be continued.  First let's read in the data and understand the types of data.  

In [2]:
df1 = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')

There are a million rows, and 81 columns

In [3]:
df1.shape

(1000000, 81)

One of these columns is a row id - so we can remove this in later processing.

In [4]:
df1.head()

,row_id,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


# Next let's understand what columns have missing values
We can quickly see from the graph that some columns have no missing values - whilst if there are missing values, these tend to be around 18k out of 1m rows.

In [5]:
missingbycol = []
colnames = df1.columns

for i in colnames:
     missingbycol.append(df1[i].isna().sum())
    
zipped = zip(colnames, missingbycol)

df2 = pd.DataFrame(zipped, columns = ['Column name', 'Number of missing values'])
df2.drop(0,axis = 0, inplace = True)
    

In [6]:
px.bar(df2, x = 'Column name', y = 'Number of missing values')

The columns with no missing values are as follows. 

In [7]:
nomissingvalues = df2[df2['Number of missing values'] ==0]['Column name']
print(nomissingvalues.count())
print(nomissingvalues)

25
16     F_2_0
17     F_2_1
18     F_2_2
19     F_2_3
20     F_2_4
21     F_2_5
22     F_2_6
23     F_2_7
24     F_2_8
25     F_2_9
26    F_2_10
27    F_2_11
28    F_2_12
29    F_2_13
30    F_2_14
31    F_2_15
32    F_2_16
33    F_2_17
34    F_2_18
35    F_2_19
36    F_2_20
37    F_2_21
38    F_2_22
39    F_2_23
40    F_2_24
Name: Column name, dtype: object


These are the columns with missing values that need to be processed. 

In [8]:
missingvalues = df2[df2['Number of missing values'] >0]['Column name']
print(missingvalues.count())
print(missingvalues)

55
1      F_1_0
2      F_1_1
3      F_1_2
4      F_1_3
5      F_1_4
6      F_1_5
7      F_1_6
8      F_1_7
9      F_1_8
10     F_1_9
11    F_1_10
12    F_1_11
13    F_1_12
14    F_1_13
15    F_1_14
41     F_3_0
42     F_3_1
43     F_3_2
44     F_3_3
45     F_3_4
46     F_3_5
47     F_3_6
48     F_3_7
49     F_3_8
50     F_3_9
51    F_3_10
52    F_3_11
53    F_3_12
54    F_3_13
55    F_3_14
56    F_3_15
57    F_3_16
58    F_3_17
59    F_3_18
60    F_3_19
61    F_3_20
62    F_3_21
63    F_3_22
64    F_3_23
65    F_3_24
66     F_4_0
67     F_4_1
68     F_4_2
69     F_4_3
70     F_4_4
71     F_4_5
72     F_4_6
73     F_4_7
74     F_4_8
75     F_4_9
76    F_4_10
77    F_4_11
78    F_4_12
79    F_4_13
80    F_4_14
Name: Column name, dtype: object


To be continued...